In [1]:
from global_search_recommendation.BioInspiredRecomendation import BioInspiredRecomendation
from percentile_recommendation.current_calculator import RecommendationCalculator
from SheetsConversor import SheetsConversor
from Complete import CompleteCalculator
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import STL
import pandas as pd

import warnings
import time

from commons.Tariff import BlueTariff, GreenTariff

In [2]:
conversor = SheetsConversor('../sheets-plus/')
ucs, mods, contract = conversor.convert()

Reading ../sheets-plus/UFPR.xlsx...
Reading ../sheets-plus/UFMG_II.xlsx...
Reading ../sheets-plus/UFRA.xlsx...
Reading ../sheets-plus/UFMG_I.xlsx...
Reading ../sheets-plus/UFFS.xlsx...
Reading ../sheets-plus/UFAL.xlsx...
Reading ../sheets-plus/UFSJ.xlsx...
Reading ../sheets-plus/UFCA.xlsx...
Reading ../sheets-plus/UFJF.xlsx...
Reading ../sheets-plus/UFOP.xlsx...
Reading ../sheets-plus/UFMS.xlsx...
Reading ../sheets-plus/UFRGS.xlsx...
Done! 130 UCs conveted!


In [5]:
from alive_progress import alive_bar
RESULT_HEADERS = ['method', 'flag', 'peak', 'off_peak', 'unique', 'cost', 'num_months', 'time']

def do_calculations(n: int, it_only=False):

    results = {}

    with alive_bar(len(ucs), force_tty=True) as bar:
        for k in ucs:
            result_frame = pd.DataFrame(columns=RESULT_HEADERS)

            cm = RecommendationCalculator(ucs[k].head(n), mods[k], BlueTariff(), GreenTariff(), n)
            result_frame.loc[len(result_frame)] = cm.calculate()
            
            if not it_only:
                cm = RecommendationCalculator(ucs[k].head(n), mods[k], BlueTariff(), GreenTariff(), n)
                result_frame.loc[len(result_frame)] = cm.calculate()

            bio_inspired = BioInspiredRecomendation(n, pd.DataFrame(ucs[k]), num_pop=10)

            if not it_only:
                result_frame.loc[len(result_frame)] = bio_inspired.do_PSO()
                result_frame.loc[len(result_frame)] = bio_inspired.do_GA()
            
            result_frame.loc[len(result_frame)] = bio_inspired.do_PSO(iterative=True)
            result_frame.loc[len(result_frame)] = bio_inspired.do_GA(iterative=True)

            if not it_only:
                complete_calculator = CompleteCalculator(n, ucs[k])
                result_frame.loc[len(result_frame)] = complete_calculator.run()
    
            results[k] = result_frame
            bar()
            
    return results

In [6]:
def get_methods_time_perf(results):
    headers = results['../sheets-plus/UFMS.xlsx - 109001794-8'].method
    times = pd.DataFrame(columns=headers)

    for r in results:
        result = results[r].drop(columns=['method', 'flag', 'peak', 'off_peak', 'unique', 'cost', 'num_months'])
        times.loc[len(times)] = result.T.values[0]
    
    return [ {h:times[h].mean()} for h in headers ]

In [5]:
warnings.filterwarnings('ignore')

results_per_range = []
for n in range(6, 13):
    print(f"Calculating to {n} months...", flush=True)
    results_per_range.append(do_calculations(n))

Calculating to 6 months...
|████████████████████████████████████████| 130/130 [100%] in 17:14.6 (0.13/s)   ▆ 0/130 [0%] in 0s (~0s, 0.0/s) 
Calculating to 7 months...
|████████████████████████████████████████| 130/130 [100%] in 26:46.7 (0.08/s)   
Calculating to 8 months...
|████████████████████████████████████████| 130/130 [100%] in 27:07.7 (0.08/s)   
Calculating to 9 months...
|████████████████████████████████████████| 130/130 [100%] in 19:59.0 (0.11/s)   
Calculating to 10 months...
|████████████████████████████████████████| 130/130 [100%] in 28:12.2 (0.08/s)   
Calculating to 11 months...
|████████████████████████████████████████| 130/130 [100%] in 28:51.8 (0.07/s)   ▆ 0/130 [0%] in 4s (~0s, 0.0/s)  ▇▇▅ 0/130 [0%] in 9s (~0s, 0.0/s) 
Calculating to 12 months...
|████████████████████████████████████████| 130/130 [100%] in 22:27.1 (0.10/s)   


In [6]:
times = {}
n = 6
for r in results_per_range:
    times.update({n:get_methods_time_perf(r)})
    n += 1

print('Médias de tempo de execução (segundos)\n')
for i in range (6, 13):
    print(f'\tUtilizando {i} meses de histórico')
    time = times[i]
    print(f"\t\tPercentil: {time[0]['Percentil']:.10f}")
    print(f"\t\tPSO:       {time[1]['PSO']:.2f}")
    print(f"\t\tGA:        {time[2]['GA']:.2f}")
    print(f"\t\tPSO-it:    {time[3]['PSO-it']:.2f}")
    print(f"\t\tGA-it:     {time[4]['GA-it']:.2f}")
    print(f"\t\tCompleto:  {time[5]['Completo']:.2f}\n")

Médias de tempo de execução (segundos)

	Utilizando 6 meses de histórico
		Percentil: 0.0000405458
		PSO:       3.46
		GA:        1.50
		PSO-it:    0.13
		GA-it:     0.08
		Completo:  2.71

	Utilizando 7 meses de histórico
		Percentil: 0.0000401772
		PSO:       7.07
		GA:        1.87
		PSO-it:    0.21
		GA-it:     0.10
		Completo:  3.03

	Utilizando 8 meses de histórico
		Percentil: 0.0000405660
		PSO:       6.99
		GA:        1.86
		PSO-it:    0.22
		GA-it:     0.11
		Completo:  3.27

	Utilizando 9 meses de histórico
		Percentil: 0.0000393042
		PSO:       3.82
		GA:        1.55
		PSO-it:    0.17
		GA-it:     0.09
		Completo:  3.51

	Utilizando 10 meses de histórico
		Percentil: 0.0000405843
		PSO:       6.93
		GA:        1.83
		PSO-it:    0.25
		GA-it:     0.12
		Completo:  3.81

	Utilizando 11 meses de histórico
		Percentil: 0.0000396563
		PSO:       6.91
		GA:        1.86
		PSO-it:    0.25
		GA-it:     0.12
		Completo:  4.10

	Utilizando 12 meses de histórico
		Percentil: 0.000040006

In [7]:
warnings.filterwarnings('ignore')

results_per_range = []
for n in range(6, 13):
    print(f"Calculating to {n} months...", flush=True)
    results_per_range.append(do_calculations(n, it_only=True))

Calculating to 6 months...
|████████████████████████████████████████| 130/130 [100%] in 21.3s (6.09/s)     
Calculating to 7 months...
|████████████████████████████████████████| 130/130 [100%] in 21.8s (5.96/s)     
Calculating to 8 months...
|████████████████████████████████████████| 130/130 [100%] in 22.7s (5.73/s)     
Calculating to 9 months...
|████████████████████████████████████████| 130/130 [100%] in 24.4s (5.30/s)     
Calculating to 10 months...
|████████████████████████████████████████| 130/130 [100%] in 24.1s (5.38/s)     
Calculating to 11 months...
|████████████████████████████████████████| 130/130 [100%] in 24.9s (5.20/s)     
Calculating to 12 months...
|████████████████████████████████████████| 130/130 [100%] in 26.1s (4.96/s)     


In [8]:
times = {}
n = 6
for r in results_per_range:
    times.update({n:get_methods_time_perf(r)})
    n += 1

print('Médias de tempo de execução (segundos)\n')
for i in range (6, 13):
    print(f'\tUtilizando {i} meses de histórico')
    time = times[i]
    print(f"\t\tPSO-it:    {time[0]['PSO-it']:.2f}")
    print(f"\t\tGA-it:     {time[1]['GA-it']:.2f}")

Médias de tempo de execução (segundos)

	Utilizando 6 meses de histórico
		PSO-it:    0.18
		GA-it:     0.10
	Utilizando 7 meses de histórico
		PSO-it:    0.19
		GA-it:     0.10
	Utilizando 8 meses de histórico
		PSO-it:    0.21
		GA-it:     0.10
	Utilizando 9 meses de histórico
		PSO-it:    0.23
		GA-it:     0.12
	Utilizando 10 meses de histórico
		PSO-it:    0.26
		GA-it:     0.12
	Utilizando 11 meses de histórico
		PSO-it:    0.26
		GA-it:     0.12
	Utilizando 12 meses de histórico
		PSO-it:    0.28
		GA-it:     0.14


In [10]:
times = {}
n = 6
for r in results_per_range:
    times.update({n:get_methods_time_perf(r)})
    n += 1

print('Médias de tempo de execução (milisegundos)\n')
for i in range (6, 13):
    print(f'\tUtilizando {i} meses de histórico')
    time = times[i]
    #print(time)
    print(f"\t\tpercentil: {time[0]['Percentil']*100:.2f}")
    print(f"\t\tPSO-it:    {time[1]['PSO-it']*100:.2f}")
    print(f"\t\tGA-it:     {time[2]['GA-it']*100:.2f}")

Médias de tempo de execução (milisegundos)

	Utilizando 6 meses de histórico
		percentil: 7.23
		PSO-it:    4.75
		GA-it:     3.98
	Utilizando 7 meses de histórico
		percentil: 7.35
		PSO-it:    5.00
		GA-it:     3.96
	Utilizando 8 meses de histórico
		percentil: 7.38
		PSO-it:    5.31
		GA-it:     4.31
	Utilizando 9 meses de histórico
		percentil: 7.50
		PSO-it:    5.90
		GA-it:     4.95
	Utilizando 10 meses de histórico
		percentil: 7.33
		PSO-it:    6.20
		GA-it:     4.58
	Utilizando 11 meses de histórico
		percentil: 7.35
		PSO-it:    6.34
		GA-it:     5.04
	Utilizando 12 meses de histórico
		percentil: 7.48
		PSO-it:    6.79
		GA-it:     5.39
